# Debugging API calls

In [1]:
from fastapi import APIRouter, UploadFile, File
import asyncio
import io
import cv2
import numpy as np
from PIL import Image, ExifTags
from app.backend.utlis.ocr import pan_ocr, adhar_ocr
from app.backend.utlis.face import extract_face, enhance_image, match_faces_with_facenet
from app.backend.utlis.logging_config import get_logger  # Import the global logger
from typing import Optional, Dict
from scipy.spatial.distance import cosine


2025-02-10 18:32:15 - INFO - [models.py:19] - models are initialized
INFO:app.backend.utlis.models:models are initialized


In [2]:
logger = get_logger(__name__)

In [3]:
def read_image(image_data: bytes) -> Optional[np.ndarray]:
    logger.info(f"Received image data, length: {len(image_data)} bytes")
    if not image_data:
        logger.error("Image data is empty or unreadable")
        return None
    try:
        image = Image.open(io.BytesIO(image_data))
        try:
            exif = image._getexif()
            if exif:
                for tag, value in exif.items():
                    if ExifTags.TAGS.get(tag) == "Orientation":
                        if value == 3:
                            image = image.rotate(180, expand=True)
                        elif value == 6:
                            image = image.rotate(270, expand=True)
                        elif value == 8:
                            image = image.rotate(90, expand=True)
        except Exception as exif_error:
            logger.warning(f"EXIF data not found or could not be processed: {exif_error}")
        image = image.convert("RGB")  # Ensure no alpha channel
        cv2_img = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        logger.info(f"Image processed successfully with shape {cv2_img.shape} in BGR format")
        return cv2_img
    except Exception as e:
        logger.error(f"Error processing image: {e}")
        return None

In [4]:
def process_documents() -> Dict[str, dict]:
    logger.info("Processing documents started")
    try:
#loading images
        with open("pan.jpg", "rb") as image_file:
            pan_image_data = image_file.read()
        with open("live.jpg", "rb") as image_file:
            live_image_data = image_file.read()
        with open("adhar_front.jpg", "rb") as image_file:
            adhar_image_data = image_file.read()
    except Exception as e:
        logger.error(f"Error reading images: {e}")
        return {"error": "Failed to read images"}
    try:
# read_image(): converting images from bytes to np.ndarray
        pan_img = read_image(pan_image_data)
        adhar_img = read_image(adhar_image_data)
        live_img = read_image(live_image_data)
    except Exception as e:
        logger.error(f"Error reading image data: {e}")
        return {"error": "Error processing image data"}
# extract_face: esrGan,mtcnn:
    #extract live_face using esrGan and then mtcnn
    extracted_face = extract_face(live_img)
    if extracted_face is None:
        logger.warning("No face detected in live image")
        return {"error": "No face detected in live image"}
    logger.info("Face successfully extracted from live image")
    try:
# performing OCR:
        pan_text= pan_ocr(image=pan_img)
        adhar_text=adhar_ocr(image=adhar_img)
        logger.info("OCR completed successfully")
    except Exception as e:
        logger.error(f"OCR processing error: {e}")
        return {"error": "Failed to perform OCR"}
# extract_face: esrGan,mtcnn:
    #extract pan_face & adhar_face using esrGan and then mtcnn
    adhar_face=extract_face(adhar_img)
    pan_face =extract_face(pan_img)
    if adhar_face is not None:
        logger.info("Face extracted from Aadhar image")
    else:
        logger.warning("No face detected in Aadhar image")
    if pan_face is not None:
        logger.info("Face extracted from PAN image")
    else:
        logger.warning("No face detected in PAN image")
# facenet: match_faces_with_facenet(): calculate face simmilarity score
    match_adhar, score_adhar = match_faces_with_facenet(extracted_face, adhar_face) if adhar_face is not None else (False, None)
    match_pan, score_pan = match_faces_with_facenet(extracted_face, pan_face) if pan_face is not None else (False, None)
    logger.info(f"Face match results - Aadhar: {match_adhar}, Score: {score_adhar}")
    logger.info(f"Face match results - PAN: {match_pan}, Score: {score_pan}")
    return {
        "pan_ocr": pan_text,
        "adhar_ocr": adhar_text,
        "face_match_with_adhar": {"match": match_adhar, "score": score_adhar},
        "face_match_with_pan": {"match": match_pan, "score": score_pan},
        "extracted_pan_face" : pan_face,
        "extracted_adhar_face" : adhar_face,
        "pan_img" : pan_img,
        "adhar_img" : adhar_img,
        "live_img" : live_img,
        "extracted_live_face" : extracted_face,
    }

In [ ]:
result=process_documents()

In [21]:
from PIL import Image
import numpy as np
arr_rgb = cv2.cvtColor(result['extracted_live_face'], cv2.COLOR_BGR2RGB)
image = Image.fromarray(arr_rgb)
image.show()

# extract face from id

In [144]:
from typing import Optional
from mtcnn import MTCNN
import cv2
import os
import numpy as np


def extract_and_save_face(image_path: str, *, save_path: str = "cropped_faces/", padding: float = 0.35) -> bool:
    """
    Detects a face in the given image, applies padding to the cropped face, saves it, and returns whether a face was found.

    Args:
        image_path (str): Path to the input image.
        save_path (str, keyword-only): Directory where cropped face images will be saved.
        padding (float, keyword-only): Padding factor (0.1 = 10% extra around the face).

    Returns:
        bool: True if a face was found and saved, False otherwise.
    """
    # Load image
    image: Optional[np.ndarray] = cv2.imread(image_path)
    if image is None:
        print(f"Error: Unable to read image {image_path}")
        return False

    # Initialize MTCNN detector
    detector = MTCNN()

    # Detect faces
    faces = detector.detect_faces(image)

    if not faces:
        print("No face detected.")
        return False

    # Ensure save directory exists
    os.makedirs(save_path, exist_ok=True)

    height, width, _ = image.shape  # Image dimensions

    for i, face in enumerate(faces):  # Loop in case there are multiple faces
        x, y, w, h = face["box"]
        print(f"x :{x}, y : {y}, w : {w}, h : {h}")

        # Compute padding
        pad_x = int(w * padding)
        pad_y = int(h * padding)

        # Expand bounding box
        x1, y1 = max(x - pad_x, 0), max(y - pad_y, 0)
        x2, y2 = min(x + w + pad_x, width), min(y + h + pad_y, height)

        # Crop the padded face
        cropped_face = image[y1:y2, x1:x2]

        # Save cropped face
        #face_filename = os.path.join(save_path, f"face_{os.path.basename(image_path)}")
        face_filename = os.path.join(save_path, f"face_{os.path.basename(image_path).replace('.jpg', '.png')}")
        cv2.imwrite(face_filename, cropped_face)
        print(f"✅ Face saved at: {face_filename} with padding {padding*100:.1f}%")
        return True  # Return True after saving the first detected face

    return False  # If no face was saved

In [146]:
success = extract_and_save_face("adhar_front.jpg")
print(f"Face extraction successful: {success}")

x :84, y : 102, w : 44, h : 55
✅ Face saved at: cropped_faces/face_adhar_front.png with padding 35.0%
Face extraction successful: True


In [147]:
success = extract_and_save_face("pan.jpg")
print(f"Face extraction successful: {success}")

x :161, y : 425, w : 124, h : 147
✅ Face saved at: cropped_faces/face_pan.png with padding 35.0%
Face extraction successful: True


In [148]:
success = extract_and_save_face("live.jpg")
print(f"Face extraction successful: {success}")

x :462, y : 103, w : 381, h : 467
✅ Face saved at: cropped_faces/face_live.png with padding 35.0%
Face extraction successful: True


# esrGAN

In [58]:
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import os


# Load Pretrained ESRGAN Model from TensorFlow Hub
srgan_model = hub.load("https://tfhub.dev/captain-pool/esrgan-tf2/1")


def enhance_image(image_path: str, *, save_path: str = "enhanced_faces/") -> bool:
    """
    Enhances the quality of a face image using a Super-Resolution GAN (SRGAN).
    Args:
        image_path (str): Path to the cropped face image.
        save_path (str, keyword-only): Directory where enhanced images will be saved.
    Returns:
        bool: True if enhancement is successful, False otherwise.
    """
    # Load image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Unable to read image {image_path}")
        return False
    # Convert to RGB and normalize (0-1)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) / 255.0
    #image/=255.0
    # Resize to match ESRGAN input requirements
    #image_resized = cv2.resize(image, (50, 50))  # Model expects 50x50 input
    image_resized = np.expand_dims(image, axis=0).astype(np.float32)
    # Run through the ESRGAN model
    enhanced = srgan_model(image_resized)[0]  # Get first output
    # ✅ Convert EagerTensor to NumPy before processing
    enhanced = np.array(enhanced)  
    enhanced = (enhanced * 255.0).clip(0, 255).astype(np.uint8)  # Process as usual
    # Ensure save directory exists
    os.makedirs(save_path, exist_ok=True)
    # Save enhanced image
    enhanced_filename = os.path.join(save_path, f"enhanced_{os.path.basename(image_path)}")
    cv2.imwrite(enhanced_filename, cv2.cvtColor(enhanced, cv2.COLOR_RGB2BGR))
    print(f"✅ Enhanced face saved at: {enhanced_filename}")
    return True

In [158]:
success = enhance_image("cropped_faces/face_live.png")
print(f"Super-resolution enhancement successful: {success}")

✅ Enhanced face saved at: enhanced_faces/enhanced_face_live.png
Super-resolution enhancement successful: True


In [159]:
success = enhance_image("cropped_faces/face_adhar_front.png")
print(f"Super-resolution enhancement successful: {success}")

✅ Enhanced face saved at: enhanced_faces/enhanced_face_adhar_front.png
Super-resolution enhancement successful: True


In [160]:
success = enhance_image("cropped_faces/face_pan.png")
print(f"Super-resolution enhancement successful: {success}")

✅ Enhanced face saved at: enhanced_faces/enhanced_face_pan.png
Super-resolution enhancement successful: True


# Face Matching- FaceNet

In [66]:
from keras_facenet import FaceNet
import numpy as np
import cv2
from scipy.spatial.distance import cosine

def match_faces_with_facenet(image1_path: str, image2_path: str):
    """
    Compares two pre-processed faces and returns whether they match and the similarity score.

    Args:
        image1_path (str): Path to the first image (already cropped and enhanced).
        image2_path (str): Path to the second image (already cropped and enhanced).

    Returns:
        tuple: A tuple containing:
            - bool: True if the faces match, False otherwise.
            - float: The cosine similarity score between the two faces.
    """
    # Load the images using OpenCV (ensure they are valid)
    image1 = cv2.imread(image1_path)
    image2 = cv2.imread(image2_path)
    
    if image1 is None:
        raise ValueError(f"Failed to load image at {image1_path}")
    if image2 is None:
        raise ValueError(f"Failed to load image at {image2_path}")

    # Resize the images to the size that FaceNet expects (160x160)
    image1 = cv2.resize(image1, (160, 160))
    image2 = cv2.resize(image2, (160, 160))

    # Convert images to RGB (FaceNet expects RGB images, OpenCV loads in BGR by default)
    image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
    image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)

    # Add an extra batch dimension: from (160, 160, 3) to (1, 160, 160, 3)
    image1 = np.expand_dims(image1, axis=0)
    image2 = np.expand_dims(image2, axis=0)

    # Initialize the FaceNet model
    facenet = FaceNet()

    # Extract face embeddings
    embedding1 = facenet.embeddings(image1)[0]
    embedding2 = facenet.embeddings(image2)[0]

    # Compute the cosine distance between the two embeddings
    distance = cosine(embedding1, embedding2)

    # Define a threshold for matching (typically, 0.6 to 0.7 is a common threshold)
    threshold = 0.6

    # Determine if faces match
    faces_match = distance < threshold

    return faces_match, distance

In [178]:
# Example usage:
image1_path = "enhanced_faces/enhanced_face_pan.png"
image2_path = "enhanced_faces/enhanced_face_live.png"

faces_match, similarity_score = match_faces_with_facenet(image1_path, image2_path)

print(f"Do the faces match? {faces_match}")
print(f"Similarity Score (distance): {similarity_score}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
Do the faces match? True
Similarity Score (distance): 0.4027039408683777


In [180]:
# Example usage:
image1_path = "enhanced_faces/enhanced_face_adhar_front.jpg"
image2_path = "enhanced_faces/enhanced_face_live.jpg"

faces_match, similarity_score = match_faces_with_facenet(image1_path, image2_path)

print(f"Do the faces match? {faces_match}")
print(f"Similarity Score (distance): {similarity_score}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
Do the faces match? True
Similarity Score (distance): 0.5981652736663818


# OCR using easyocr 

### PAN

In [3]:
import cv2
import numpy as np
import easyocr
import re
import logging

# Configure logging (PEP 282)
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")


def preprocess_image(image_path: str) -> np.ndarray:
    """
    Preprocess the image by converting it to grayscale and applying adaptive thresholding.
    
    Args:
        image_path (str): Path to the image file.

    Returns:
        np.ndarray: Preprocessed thresholded image.
    """
    img = cv2.imread(image_path)

    if img is None:
        logging.error(f"Failed to load image: {image_path}")
        raise FileNotFoundError(f"Image not found at {image_path}")

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply adaptive thresholding to enhance text visibility
    thresh = cv2.adaptiveThreshold(
        gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2
    )

    logging.info("Image preprocessing completed.")
    return thresh


def extract_pan_details(*, image_path: str) -> dict:
    """
    Extract important details from a PAN card image using OCR.

    Args:
        image_path (str): Path to the image file.

    Returns:
        dict: Extracted information including ID Number, Date of Birth, Name, and Father's Name.
    """
    reader = easyocr.Reader(["en"])  # Load OCR reader for English

    # Preprocess the image
    processed_img = preprocess_image(image_path)

    # Extract text using EasyOCR
    results = reader.readtext(processed_img, detail=0)  # Extract text only

    extracted_info = {
        "ID Number": None,
        "Date of Birth": None,
        "Name": None,
        "Father's Name": None,
    }

    name_found = False  # Track if Name is found
    father_name_found = False  # Track if Father's Name is found

    for i, text in enumerate(results):
        text = text.strip()

        # Extract PAN Number (Format: ABCDE1234F)
        if re.match(r"^[A-Z]{5}[0-9]{4}[A-Z]$", text):
            extracted_info["ID Number"] = text

        # Extract Date of Birth (Format: DD/MM/YYYY)
        if re.match(r"\d{2}/\d{2}/\d{4}", text):
            extracted_info["Date of Birth"] = text

        # Extract Name (Assuming it appears after "Name" keyword)
        if "name" in text.lower() and i + 1 < len(results) and not name_found:
            extracted_info["Name"] = results[i + 1].strip()
            name_found = True

        # Extract Father's Name (Appears after "Father's Name" or "पिता का नाम")
        if "father" in text.lower() and i + 1 < len(results) and not father_name_found:
            extracted_info["Father's Name"] = results[i + 1].strip()
            father_name_found = True

    logging.info(f"Extracted details: {extracted_info}")
    return extracted_info

### adhar card front

In [24]:
def ext(image_path):
    """
    Extract important details from an Aadhaar card image using OCR.

    Args:
        image_path (str): Path to the image file.

    Returns:
        dict: Extracted information including Aadhaar Number, Name, Date of Birth, and Gender.
    """
    reader = easyocr.Reader(["en"])  # Load OCR reader for English & Hindi

    # Preprocess the image
    processed_img = preprocess_image(image_path)

    # Extract text using EasyOCR
    results = reader.readtext(processed_img, detail=0)  # Extract text only

    extracted_info = {
        "Aadhaar Number": None,
        "Date of Birth": None,
        "Gender": None,
    }

    name_candidates = []  # Track if Name is found
    gender_found = False  # Track if Gender is found
    
    for i, text in enumerate(results):
        text = text.strip()

        # Extract Aadhaar Number (Format: 12-digit number)
        if re.match(r"^\d{4}\s?\d{4}\s?\d{4}$", text):
            extracted_info["Aadhaar Number"] = text.replace(" ", "")  # Remove spaces

        # Extract Date of Birth (Format: DD/MM/YYYY)
        match=re.search(r"\b(\d{2}/\d{2}/\d{4})\b", text)
        if match:
            extracted_info["Date of Birth"] = match.group(1)



        # Extract Gender (Look for "Male", "Female" or "पुरुष", "महिला")
        if not gender_found:
            if "male" in text.lower():
                extracted_info["Gender"] = "Male"
                gender_found = True
            elif "female" in text.lower():
                extracted_info["Gender"] = "Female"
                gender_found = True

    return extracted_info



In [109]:
res=extract_pan_details(image_path="pan.jpg")

# DEBUGGING

In [ ]:
#load images
# read_image()
#extract_face() OR esrGAN and mtcnn
#OCR
#facenet

###### read jpg files into bytes

In [4]:
with open("pan.jpg", "rb") as image_file:
    pan_image_data = image_file.read()

with open("live.jpg", "rb") as image_file:
    live_image_data = image_file.read()

with open("adhar_front.jpg", "rb") as image_file:
    adhar_image_data = image_file.read()

In [5]:
pan=read_image(pan_image_data)
live=read_image(live_image_data)
adhar=read_image(adhar_image_data)

2025-02-09 21:35:27 - INFO - [486770899.py:2] - Received image data, length: 866002 bytes
INFO:__main__:Received image data, length: 866002 bytes
2025-02-09 21:35:27 - INFO - [486770899.py:23] - Image processed successfully with shape (1241, 1754, 3)
INFO:__main__:Image processed successfully with shape (1241, 1754, 3)
2025-02-09 21:35:27 - INFO - [486770899.py:2] - Received image data, length: 126576 bytes
INFO:__main__:Received image data, length: 126576 bytes
2025-02-09 21:35:27 - INFO - [486770899.py:23] - Image processed successfully with shape (720, 1280, 3)
INFO:__main__:Image processed successfully with shape (720, 1280, 3)
2025-02-09 21:35:27 - INFO - [486770899.py:2] - Received image data, length: 55807 bytes
INFO:__main__:Received image data, length: 55807 bytes
2025-02-09 21:35:27 - INFO - [486770899.py:23] - Image processed successfully with shape (344, 543, 3)
INFO:__main__:Image processed successfully with shape (344, 543, 3)


###### using notebook functions

In [41]:
#from typing import Optional
def extract_and_save_face(image: np.ndarray , padding: float = 0.35) -> Optional[np.ndarray]:
    if image is None:
        print(f"Error: Unable to read image")
        return False
    faces = detector.detect_faces(image)
    if not faces:
        print("No face detected.")
        return False
    height, width, _ = image.shape 
    for i, face in enumerate(faces): 
        x, y, w, h = face["box"] 
        print(f"x :{x}, y : {y}, w : {w}, h : {h}")
        pad_x = int(w * padding)
        pad_y = int(h * padding)
        x1, y1 = max(x - pad_x, 0), max(y - pad_y, 0)
        x2, y2 = min(x + w + pad_x, width), min(y + h + pad_y, height)
        cropped_face = image[y1:y2, x1:x2]
        return cropped_face
    return None 
def enhance_image(image: np.ndarray) -> Optional[np.ndarray]:
    if image is None:
        print(f"Error: Unable to read image")
        return False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) / 255.0
    image_resized = np.expand_dims(image, axis=0).astype(np.float32)
    enhanced = srgan_model(image_resized)[0]  
    enhanced = np.array(enhanced)  
    enhanced = (enhanced * 255.0).clip(0, 255).astype(np.uint8)  # Process as usual
    enh=cv2.cvtColor(enhanced, cv2.COLOR_RGB2BGR)
    return enh
def match_faces_with_facenet(image1: np.ndarray, image2: np.ndarray):
    if image1 is None:
        raise ValueError(f"Failed to load image")
    if image2 is None:
        raise ValueError(f"Failed to load image")
    image1 = cv2.resize(image1, (160, 160))
    image2 = cv2.resize(image2, (160, 160))
    image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
    image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
    image1 = np.expand_dims(image1, axis=0)
    image2 = np.expand_dims(image2, axis=0)
    embedding1 = facenet.embeddings(image1)[0]
    embedding2 = facenet.embeddings(image2)[0]
    distance = cosine(embedding1, embedding2)
    threshold = 0.6
    faces_match = distance < threshold
    return faces_match, distance

In [13]:
from app.backend.utlis.models import mtcnn_detector as detector
from app.backend.utlis.models import srgan_model
from app.backend.utlis.models import facenet
from app.backend.utlis.logging_config import get_logger

###### using extract_face()

In [15]:
extract_pan=extract_face(pan)
extract_live=extract_face(live)
extract_adhar=extract_face(adhar)

x :161, y : 425, w : 124, h : 147


2025-02-09 21:36:04 - INFO - [face.py:96] - Image enhancement successful.
INFO:app.backend.utlis.face:Image enhancement successful.
2025-02-09 21:36:04 - INFO - [face.py:62] - Face extracted and enhanced with padding 35.0%
INFO:app.backend.utlis.face:Face extracted and enhanced with padding 35.0%


x :462, y : 103, w : 381, h : 467


2025-02-09 21:36:30 - INFO - [face.py:96] - Image enhancement successful.
INFO:app.backend.utlis.face:Image enhancement successful.
2025-02-09 21:36:30 - INFO - [face.py:62] - Face extracted and enhanced with padding 35.0%
INFO:app.backend.utlis.face:Face extracted and enhanced with padding 35.0%


x :84, y : 102, w : 44, h : 55


2025-02-09 21:36:32 - INFO - [face.py:96] - Image enhancement successful.
INFO:app.backend.utlis.face:Image enhancement successful.
2025-02-09 21:36:32 - INFO - [face.py:62] - Face extracted and enhanced with padding 35.0%
INFO:app.backend.utlis.face:Face extracted and enhanced with padding 35.0%


###### notbook files

In [16]:
ext_pan=extract_and_save_face(pan)
ext_live=extract_and_save_face(live)
ext_adhar=extract_and_save_face(adhar)

x :161, y : 425, w : 124, h : 147
x :462, y : 103, w : 381, h : 467
x :84, y : 102, w : 44, h : 55


In [17]:
enh_pan=enhance_image(ext_pan)
enh_live=enhance_image(ext_live)
enh_adhar=enhance_image(ext_adhar)

##### checking if the reult from two different ways are same or not

###### Shape

In [18]:
print("shape of PAN")
print(f"extracted face dimensions from notebook : {ext_pan.shape} and type : {type(ext_pan)}")
print(f"enhanced face dimensions from notebook : {enh_pan.shape} and type : {type(enh_pan)}")
print(f"extracted & enhanced face dimensions : {extract_pan.shape} and type : {type(extract_pan)}")

shape of PAN
extracted face dimensions from notebook : (249, 210, 3) and type : <class 'numpy.ndarray'>
enhanced face dimensions from notebook : (996, 840, 3) and type : <class 'numpy.ndarray'>
extracted & enhanced face dimensions : (996, 840, 3) and type : <class 'numpy.ndarray'>


In [19]:
print("shape of adhar")
print(f"extracted face dimensions from notebook : {ext_adhar.shape}")
print(f"enhanced face dimensions from notebook : {enh_adhar.shape}")
print(f"extracted & enhanced face dimensions : {extract_adhar.shape}")

shape of adhar
extracted face dimensions from notebook : (93, 74, 3)
enhanced face dimensions from notebook : (372, 296, 3)
extracted & enhanced face dimensions : (372, 296, 3)


In [20]:
print("shape of live")
print(f"extracted face dimensions from notebook : {ext_live.shape}")
print(f"enhanced face dimensions from notebook : {enh_live.shape}")
print(f"extracted & enhanced face dimensions : {extract_live.shape}")

shape of live
extracted face dimensions from notebook : (720, 647, 3)
enhanced face dimensions from notebook : (2880, 2588, 3)
extracted & enhanced face dimensions : (2880, 2588, 3)


###### if both are same or not

In [23]:
print(f"check for PAN(floating point precision) : {np.array_equal(extract_pan, enh_pan)}")
print(f"check for PAN : {np.allclose(extract_pan, enh_pan, atol=1e-8)}")
print(f"check for adhar(floating point precision) : {np.array_equal(extract_adhar, enh_adhar)}")
print(f"check for adhar : {np.allclose(extract_adhar, enh_adhar, atol=1e-8)}")
print(f"check for LIVE(floating point precision) : {np.array_equal(extract_live, enh_live)}")
print(f"check for LIVE : {np.allclose(extract_live, enh_live, atol=1e-8)}")

check for PAN(floating point precision) : True
check for PAN : True
check for adhar(floating point precision) : True
check for adhar : True
check for LIVE(floating point precision) : True
check for LIVE : True


##### debugging face matching 

###### face matching

In [43]:
panXlive=match_faces_with_facenet(enh_pan,enh_live)
adharXlive=match_faces_with_facenet(enh_adhar,enh_live)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step


In [48]:
adharXlive

(False, 0.6518564314756645)

In [72]:
panXlive

(True, 0.4027039408683777)

###### checking why reults are different

In [164]:
crp_pan = cv2.imread("cropped_faces/face_pan.png")
crp_live = cv2.imread("cropped_faces/face_live.png")
crp_adhar = cv2.imread("cropped_faces/face_adhar_front.png")
fin_pan = cv2.imread("enhanced_faces/enhanced_face_pan.png")
fin_adhar = cv2.imread("enhanced_faces/enhanced_face_adhar_front.png")
fin_live = cv2.imread("enhanced_faces/enhanced_face_live.png")

In [166]:
print("shape of PAN")
print(f"extracted face dimensions from notebook : {crp_pan.shape} and type : {type(crp_pan)}")
print(f"enhanced face dimensions from notebook : {fin_pan.shape} and type : {type(fin_pan)}")
print(f"extracted & enhanced face dimensions : {enh_pan.shape} and type : {type(enh_pan)}")

shape of PAN
extracted face dimensions from notebook : (249, 210, 3) and type : <class 'numpy.ndarray'>
enhanced face dimensions from notebook : (996, 840, 3) and type : <class 'numpy.ndarray'>
extracted & enhanced face dimensions : (996, 840, 3) and type : <class 'numpy.ndarray'>


In [168]:
print(f"check for PAN(floating point precision) : {np.array_equal(fin_pan, enh_pan)}")
print(f"check for PAN : {np.allclose(fin_pan, enh_pan, atol=1e-8)}")
print(f"check for adhar(floating point precision) : {np.array_equal(fin_adhar, enh_adhar)}")
print(f"check for adhar : {np.allclose(fin_adhar, enh_adhar, atol=1e-8)}")
print(f"check for LIVE(floating point precision) : {np.array_equal(fin_live, enh_live)}")
print(f"check for LIVE : {np.allclose(fin_live, enh_live, atol=1e-8)}")

check for PAN(floating point precision) : True
check for PAN : True
check for adhar(floating point precision) : True
check for adhar : True
check for LIVE(floating point precision) : True
check for LIVE : True


In [182]:
print(f"check for PAN(floating point precision) : {np.array_equal(crp_pan, ext_pan)}")
print(f"check for PAN : {np.allclose(crp_pan, ext_pan, atol=1e-8)}")
print(f"check for adhar(floating point precision) : {np.array_equal(crp_adhar, ext_adhar)}")
print(f"check for adhar : {np.allclose(crp_adhar, ext_adhar, atol=1e-8)}")
print(f"check for LIVE(floating point precision) : {np.array_equal(crp_live, ext_live)}")
print(f"check for LIVE : {np.allclose(crp_live, ext_live, atol=1e-8)}")

check for PAN(floating point precision) : True
check for PAN : True
check for adhar(floating point precision) : True
check for adhar : True
check for LIVE(floating point precision) : True
check for LIVE : True


In [93]:
def is_rgb(image: np.ndarray) -> bool:
    gray_bgr = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_rgb = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    return np.array_equal(gray_bgr, gray_rgb)  # If equal, original was RGB
def check(image: np.ndarray):
    if is_rgb(image):
        print("The image is in RGB format")
    else:
        print("The image is in BGR format")

In [170]:
check(enh_adhar)
check(fin_adhar)

The image is in BGR format
The image is in BGR format


In [ ]:
#PAN      x :161,  y : 425, w : 124, h : 147
#pan      x :161,  y : 425, w : 124, h : 147
#live     x :462,  y : 103, w : 381, h : 467
#adhar    x :84,   y : 102, w : 44,  h : 55


In [172]:
hist_a = cv2.calcHist([crp_adhar], [0], None, [256], [0, 256])
hist_b = cv2.calcHist([ext_adhar], [0], None, [256], [0, 256])

print(np.allclose(hist_a, hist_b, atol=1e-5))  # Should return True if only minor pixel shifts exist


True


# streamlit

In [11]:
import streamlit as st
import cv2
import numpy as np
import tempfile
import os

def main():
    st.title("KYC Webcam Capture")

    # Initialize session state for controlling webcam
    if "webcam_active" not in st.session_state:
        st.session_state.webcam_active = False
    if "captured_image" not in st.session_state:
        st.session_state.captured_image = None

    # Button to open webcam
    if not st.session_state.webcam_active:
        if st.button("Take Picture"):
            st.session_state.webcam_active = True

    # If webcam is active, open webcam
    if st.session_state.webcam_active:
        # Start webcam using OpenCV
        cap = cv2.VideoCapture(0)  # 0 for default webcam

        # Check if webcam is opened successfully
        if not cap.isOpened():
            st.error("Could not access webcam.")
            st.session_state.webcam_active = False
            return

        st.write("Press 'Capture Image' to take a snapshot.")

        # Placeholder for displaying the video feed
        frame_placeholder = st.empty()
        capture_button = st.button("Capture Image")

        while True:
            ret, frame = cap.read()
            if not ret:
                st.error("Failed to grab frame.")
                break

            # Convert OpenCV frame (BGR) to RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Display the live video feed
            frame_placeholder.image(frame_rgb, channels="RGB")

            # If "Capture Image" button is pressed, save the image
            if capture_button:
                st.session_state.captured_image = frame_rgb
                st.session_state.webcam_active = False
                break

        cap.release()  # Release the webcam

    # Display the captured image
    if st.session_state.captured_image is not None:
        st.subheader("Captured Image:")
        st.image(st.session_state.captured_image, channels="RGB")


In [ ]:
import streamlit as st
import requests

# API Endpoint (Update if different)
API_URL = "http://localhost:8000/results/proces s_documents/"

def send_images_to_api(live_image, pan_image, adhar_image):
    """Sends uploaded images to FastAPI endpoint."""
    
    # Prepare files for API request
    files = {
        "live_image": ("live.jpg", live_image, "image/jpeg"),
        "pan_image": ("pan.jpg", pan_image, "image/jpeg"),
        "adhar_image": ("adhar.jpg", adhar_image, "image/jpeg"),
    }

    response = requests.post(API_URL, files=files)

    if response.status_code == 200:
        return response.json()
    else:
        return {"error": f"API request failed with status code {response.status_code}"}

def main():
    st.title("📄 KYC Verification System")

    # Upload Images
    st.subheader("📤 Upload Required Documents")

    live_image = st.file_uploader("Upload Live Image", type=["jpg", "png", "jpeg"])
    pan_image = st.file_uploader("Upload PAN Card Image", type=["jpg", "png", "jpeg"])
    adhar_image = st.file_uploader("Upload Aadhaar Card Image", type=["jpg", "png", "jpeg"])

    # Submit Button
    if st.button("Submit for Verification"):
        if live_image and pan_image and adhar_image:
            st.write("📡 Sending images to API...")

            # Read image bytes
            response = send_images_to_api(live_image.read(), pan_image.read(), adhar_image.read())

            # Display API response
            st.subheader("✅ API Response:")
            st.json(response)
        else:
            st.error("❌ Please upload all three images before submitting.")

if __name__ == "__main__":
    main()


In [ ]:
import streamlit as st
import cv2
import numpy as np
from app.backend.utlis.logging_config import get_logger
logger=get_logger(__name__)

# Initialize session state variables
if "captured_image_bytes" not in st.session_state:
    st.session_state.captured_image_bytes = None
if "pan_upload" not in st.session_state:
    st.session_state.pan_upload = None
if "adhar_upload" not in st.session_state:
    st.session_state.adhar_upload = None
if "image_captured" not in st.session_state:
    st.session_state.image_captured = False

# Function to capture an image from webcam
def capture_image():
    logger.info("Attempting to access webcam")
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        logger.error("Webcam not found or could not be accessed")
        st.error("Webcam not accessible. Please allow camera access.")
        return

    ret, frame = cap.read()
    cap.release()

    if ret:
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        _, buffer = cv2.imencode(".jpg", frame_rgb)  # Convert to JPEG
        st.session_state.captured_image_bytes = buffer.tobytes()  # Store in session state
        st.session_state.image_captured = True  # Mark as captured
        st.image(frame_rgb, caption="Captured Image", use_column_width=True)
        logger.info("Image captured and stored in session state")
    else:
        logger.error("Failed to capture image from webcam")

# UI Layout
st.title("Document Upload Portal")

# Capture Image Button
if st.button("Capture Image"):
    capture_image()

# File uploaders
pan_upload = st.file_uploader("Upload PAN Card", type=["jpg", "png", "jpeg"])
if pan_upload:
    st.session_state.pan_upload = pan_upload
    logger.info("PAN card uploaded successfully")

adhar_upload = st.file_uploader("Upload Aadhaar Card", type=["jpg", "png", "jpeg"])
if adhar_upload:
    st.session_state.adhar_upload = adhar_upload
    logger.info("Aadhaar card uploaded successfully")

# Submit button
if st.button("Submit"):
    logger.info(f"Captured Image in session: {st.session_state.get('captured_image_bytes') is not None}")
    logger.info(f"PAN Image uploaded: {st.session_state.pan_upload is not None}")
    logger.info(f"Aadhaar Image uploaded: {st.session_state.adhar_upload is not None}")

    # Check if all images are available
    if st.session_state.get("captured_image_bytes") and st.session_state.pan_upload and st.session_state.adhar_upload:
        st.success("✅ Submission successful! All required documents uploaded.")
        logger.info("✅ Submission successful!")
    else:
        st.error("❌ Submission failed - One or more images missing")
        logger.error("❌ Submission failed - One or more images missing")


In [ ]:
import streamlit as st
import cv2
import numpy as np
import requests
from io import BytesIO
from PIL import Image
from app.logging_config import get_logger

# Initialize Logger
logger = get_logger(__name__)

# FastAPI API URL
API_URL = "http://localhost:8000/results/process_documents/"

# Initialize session state variables
if "captured_image_bytes" not in st.session_state:
    st.session_state.captured_image_bytes = None
if "pan_upload" not in st.session_state:
    st.session_state.pan_upload = None
if "adhar_upload" not in st.session_state:
    st.session_state.adhar_upload = None
if "image_captured" not in st.session_state:
    st.session_state.image_captured = False
if "submitted" not in st.session_state:
    st.session_state.submitted = False  # Prevent multiple submissions

# Function to capture an image from the webcam
def capture_image():
    logger.info("📸 Attempting to capture live image from webcam...")
    st.write("📸 Capturing image from webcam...")
    
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        st.error("❌ Webcam not accessible. Please allow camera access.")
        logger.error("❌ Webcam not accessible. Check permissions or connection.")
        return

    ret, frame = cap.read()
    cap.release()

    if ret:
        logger.info("✅ Successfully captured image from webcam.")
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        _, buffer = cv2.imencode(".jpg", frame_rgb)  # Convert to JPEG
        st.session_state.captured_image_bytes = buffer.tobytes()  # Store in session state
        st.session_state.image_captured = True  # Mark as captured

        st.image(frame_rgb, caption="📷 Captured Live Image", use_column_width=True)
        st.success("✅ Image captured successfully!")
    else:
        st.error("❌ Failed to capture image from webcam.")
        logger.error("❌ Failed to capture image from webcam.")

# Function to send images to FastAPI backend (Synchronous)
def send_images_to_api():
    if st.session_state.submitted:
        st.warning("⚠️ Submission already in progress. Please wait...")
        logger.warning("⚠️ Submission already in progress. Preventing duplicate requests.")
        return

    logger.info("🚀 Preparing to send images to API...")

    if not st.session_state.captured_image_bytes or not st.session_state.pan_upload or not st.session_state.adhar_upload:
        st.error("❌ Submission failed - Please upload all required images")
        logger.error("❌ Missing images. Submission aborted.")
        return

    st.session_state.submitted = True  # Prevent multiple submissions

    # Convert images to file-like objects
    files = {
        "pan_image": ("pan.jpg", st.session_state.pan_upload.getvalue(), "image/jpeg"),
        "adhar_image": ("adhar.jpg", st.session_state.adhar_upload.getvalue(), "image/jpeg"),
        "live_image": ("live.jpg", BytesIO(st.session_state.captured_image_bytes), "image/jpeg"),
    }

    logger.info("📤 Sending images to FastAPI endpoint...")

    # Send request to API
    try:
        response = requests.post(API_URL, files=files)
        logger.info(f"✅ API Response Received with status code {response.status_code}")

        if response.status_code == 200:
            result = response.json()
            st.success("✅ Documents processed successfully!")

            # Display results
            st.subheader("📜 OCR Results")
            st.write(f"**PAN OCR:** {result.get('pan_ocr', 'N/A')}")
            st.write(f"**Aadhar OCR:** {result.get('adhar_ocr', 'N/A')}")

            logger.info("✅ OCR Results displayed.")

            st.subheader("🆔 Face Match Results")
            face_match_adhar = result.get("face_match_with_adhar", {})
            face_match_pan = result.get("face_match_with_pan", {})

            st.write(f"**Face Match with Aadhaar:** {'✅ Matched' if face_match_adhar.get('match') else '❌ Not Matched'}")
            st.write(f"**Match Score (Aadhaar):** {face_match_adhar.get('score', 'N/A')}")

            st.write(f"**Face Match with PAN:** {'✅ Matched' if face_match_pan.get('match') else '❌ Not Matched'}")
            st.write(f"**Match Score (PAN):** {face_match_pan.get('score', 'N/A')}")

            logger.info("✅ Face match results displayed.")

        else:
            st.error(f"❌ Failed to process documents: {response.text}")
            logger.error(f"❌ API Error: {response.text}")

    except Exception as e:
        st.error(f"❌ Error: {e}")
        logger.error(f"❌ Exception while sending request to API: {e}")

    st.session_state.submitted = False  # Reset submission state
    logger.info("🔄 Submission state reset.")

# Streamlit UI Layout
st.title("📄 Document Upload Portal")
logger.info("🎬 Streamlit UI loaded.")

# Capture Image Button
if st.button("📸 Capture Live Image"):
    logger.info("🛑 Capture Live Image button clicked.")
    capture_image()

# File uploaders for PAN and Aadhaar
pan_upload = st.file_uploader("🆔 Upload PAN Card", type=["jpg", "png", "jpeg"])
if pan_upload:
    st.session_state.pan_upload = pan_upload
    logger.info("📂 PAN image uploaded.")

adhar_upload = st.file_uploader("🆔 Upload Aadhaar Card", type=["jpg", "png", "jpeg"])
if adhar_upload:
    st.session_state.adhar_upload = adhar_upload
    logger.info("📂 Aadhaar image uploaded.")

# Submit button
if st.button("🚀 Submit to API"):
    logger.info("🛑 Submit button clicked. Processing...")
    st.write("📤 Sending data to API... Please wait.")
    send_images_to_api()  # Synchronous API call
